In [ ]:
# Import required libraries
# pandas: for data manipulation and analysis
# requests: for making HTTP requests to Binance API
# math: for mathematical operations
# Decimal: for precise decimal arithmetic
import pandas as pd
import requests
import math
from decimal import Decimal

In [ ]:
# Define the trading pair symbol and API endpoint
symbol = "SOLUSDT"  # Trading pair: Solana/USDT
url = "https://api.binance.com/api/v1/depth"  # Binance orderbook endpoint

# Set the interval for price level aggregation (0.1 USDT)
interval = Decimal('0.1')

# Parameters for the API request
# limit=1000 gets the top 1000 bids and asks
params = {
    "symbol": symbol,
    "limit": 1000,
}

In [96]:
data = requests.get(url, params).json()
print(data)

{'lastUpdateId': 25312975507, 'bids': [['187.24000000', '124.26000000'], ['187.23000000', '149.66400000'], ['187.22000000', '234.64800000'], ['187.21000000', '329.04200000'], ['187.20000000', '203.59200000'], ['187.19000000', '139.83000000'], ['187.18000000', '172.11800000'], ['187.17000000', '278.91000000'], ['187.16000000', '340.42100000'], ['187.15000000', '184.34100000'], ['187.14000000', '208.58600000'], ['187.13000000', '367.87300000'], ['187.12000000', '658.36300000'], ['187.11000000', '462.36900000'], ['187.10000000', '469.14500000'], ['187.09000000', '235.59900000'], ['187.08000000', '92.93200000'], ['187.07000000', '185.20800000'], ['187.06000000', '296.55500000'], ['187.05000000', '317.20400000'], ['187.04000000', '130.11900000'], ['187.03000000', '174.92000000'], ['187.02000000', '322.61700000'], ['187.01000000', '214.31300000'], ['187.00000000', '208.57800000'], ['186.99000000', '609.19900000'], ['186.98000000', '314.04700000'], ['186.97000000', '240.92100000'], ['186.9600

## Aggregating Bids Levels

In [ ]:
# Create a DataFrame for bid orders
# Convert the bids data from the API response into a structured DataFrame
# Each row contains: price level and quantity available at that price
bids_levels = pd.DataFrame(data['bids'], columns=['price', 'quantity'], dtype=float)
bids_levels['side'] = 'bid'  # Add a column to identify this as bid orders
print(bids_levels)
print(bids_levels.dtypes)  # Display data types of each column

      price  quantity side
0    187.24   124.260  bid
1    187.23   149.664  bid
2    187.22   234.648  bid
3    187.21   329.042  bid
4    187.20   203.592  bid
..      ...       ...  ...
995  177.29   152.012  bid
996  177.28     0.652  bid
997  177.27     3.083  bid
998  177.26   261.550  bid
999  177.25     0.927  bid

[1000 rows x 3 columns]
price       float64
quantity    float64
side         object
dtype: object


In [ ]:
# Calculate the minimum bid price level
# This rounds down the minimum bid price to the nearest interval (0.1 USDT)
min_bid_level = math.floor(min(bids_levels['price']) / float(interval)) * interval
min_bid_level

Decimal('177.2')

In [ ]:
# Calculate the maximum bid price level
# Round up to the next interval and add one more interval for binning
max_bid_level = (math.ceil(max(bids_levels['price']) / float(interval)) + 1) * interval
max_bid_level

Decimal('187.4')

In [ ]:
# Create an array of price levels for binning
# Generate price boundaries from min to max bid level with the specified interval
bid_levels_bounds = [float(min_bid_level + interval * i)
    for i in range(int((max_bid_level - min_bid_level) / interval) + 1)
]
bid_levels_bounds

[177.2,
 177.3,
 177.4,
 177.5,
 177.6,
 177.7,
 177.8,
 177.9,
 178.0,
 178.1,
 178.2,
 178.3,
 178.4,
 178.5,
 178.6,
 178.7,
 178.8,
 178.9,
 179.0,
 179.1,
 179.2,
 179.3,
 179.4,
 179.5,
 179.6,
 179.7,
 179.8,
 179.9,
 180.0,
 180.1,
 180.2,
 180.3,
 180.4,
 180.5,
 180.6,
 180.7,
 180.8,
 180.9,
 181.0,
 181.1,
 181.2,
 181.3,
 181.4,
 181.5,
 181.6,
 181.7,
 181.8,
 181.9,
 182.0,
 182.1,
 182.2,
 182.3,
 182.4,
 182.5,
 182.6,
 182.7,
 182.8,
 182.9,
 183.0,
 183.1,
 183.2,
 183.3,
 183.4,
 183.5,
 183.6,
 183.7,
 183.8,
 183.9,
 184.0,
 184.1,
 184.2,
 184.3,
 184.4,
 184.5,
 184.6,
 184.7,
 184.8,
 184.9,
 185.0,
 185.1,
 185.2,
 185.3,
 185.4,
 185.5,
 185.6,
 185.7,
 185.8,
 185.9,
 186.0,
 186.1,
 186.2,
 186.3,
 186.4,
 186.5,
 186.6,
 186.7,
 186.8,
 186.9,
 187.0,
 187.1,
 187.2,
 187.3,
 187.4]

In [ ]:
# Assign each bid price to its corresponding price bin
# right=False means intervals are left-inclusive
# precision=10 ensures we don't lose decimal precision
bids_levels["bin"] = pd.cut(
    bids_levels['price'],
    bins=bid_levels_bounds,
    right=False,
    precision=10
)
bids_levels

,price,quantity,side,bin
0,187.24,124.260,bid,"[187.2, 187.3)"
1,187.23,149.664,bid,"[187.2, 187.3)"
2,187.22,234.648,bid,"[187.2, 187.3)"
3,187.21,329.042,bid,"[187.2, 187.3)"
4,187.20,203.592,bid,"[187.2, 187.3)"
...,...,...,...,...
995,177.29,152.012,bid,"[177.2, 177.3)"
996,177.28,0.652,bid,"[177.2, 177.3)"
997,177.27,3.083,bid,"[177.2, 177.3)"
998,177.26,261.550,bid,"[177.2, 177.3)"


In [ ]:
# Group the bids by their price bins and aggregate the data
# Sum up quantities for each price level
# Keep the 'side' information (all 'bid' in this case)
# Reset index to make the bin column a regular column
bids_levels = bids_levels.groupby("bin").agg(
    quantity = ("quantity", "sum"),
    side = ("side", "first")
).reset_index()

# Extract the lower bound of each bin as the price level label
bids_levels['label'] = bids_levels['bin'].apply(lambda x: x.left)
bids_levels

C:\Users\Saksham Maggu\AppData\Local\Temp\ipykernel_2052\1399697312.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bids_levels = bids_levels.groupby("bin").agg(


,bin,quantity,side,label
0,"[177.2, 177.3)",418.224,bid,177.2
1,"[177.3, 177.4)",46.208,bid,177.3
2,"[177.4, 177.5)",89.597,bid,177.4
3,"[177.5, 177.6)",255.207,bid,177.5
4,"[177.6, 177.7)",611.574,bid,177.6
...,...,...,...,...
97,"[186.9, 187.0)",2643.024,bid,186.9
98,"[187.0, 187.1)",2178.045,bid,187.0
99,"[187.1, 187.2)",3281.956,bid,187.1
100,"[187.2, 187.3)",1041.206,bid,187.2


## Aggregating Asks Levels

In [ ]:
# Create a DataFrame for ask orders
# Convert the asks data from the API response into a structured DataFrame
# Each row contains: price level and quantity available at that price
asks_levels = pd.DataFrame(data['asks'], columns=['price', 'quantity'], dtype=float)
asks_levels['side'] = 'ask'  # Add a column to identify this as ask orders
print(asks_levels)
print(asks_levels.dtypes)  # Display data types of each column

      price  quantity side
0    187.25    74.361  ask
1    187.26     3.222  ask
2    187.27    17.383  ask
3    187.28   229.780  ask
4    187.29   299.143  ask
..      ...       ...  ...
995  197.20    33.770  ask
996  197.21     0.620  ask
997  197.22     1.638  ask
998  197.23     8.418  ask
999  197.24    10.513  ask

[1000 rows x 3 columns]
price       float64
quantity    float64
side         object
dtype: object


In [ ]:
# Calculate minimum ask level
# Round down to nearest interval and subtract one interval to ensure all asks are included
min_ask_level = (math.floor(min(asks_levels['price']) / float(interval)) - 1) * interval
min_ask_level

Decimal('187.1')

In [ ]:
# Calculate maximum ask level
# Round up to next interval and add one more interval for complete binning
max_ask_level = (math.ceil(max(asks_levels['price']) / float(interval)) + 1) * interval
max_ask_level

Decimal('197.4')

In [ ]:
# Create an array of price levels for ask binning
# Generate price boundaries from min to max ask level with the specified interval
ask_levels_bounds = [float(min_ask_level + interval * i)
    for i in range(int((max_ask_level - min_ask_level) / interval) + 1)
]
ask_levels_bounds

[187.1,
 187.2,
 187.3,
 187.4,
 187.5,
 187.6,
 187.7,
 187.8,
 187.9,
 188.0,
 188.1,
 188.2,
 188.3,
 188.4,
 188.5,
 188.6,
 188.7,
 188.8,
 188.9,
 189.0,
 189.1,
 189.2,
 189.3,
 189.4,
 189.5,
 189.6,
 189.7,
 189.8,
 189.9,
 190.0,
 190.1,
 190.2,
 190.3,
 190.4,
 190.5,
 190.6,
 190.7,
 190.8,
 190.9,
 191.0,
 191.1,
 191.2,
 191.3,
 191.4,
 191.5,
 191.6,
 191.7,
 191.8,
 191.9,
 192.0,
 192.1,
 192.2,
 192.3,
 192.4,
 192.5,
 192.6,
 192.7,
 192.8,
 192.9,
 193.0,
 193.1,
 193.2,
 193.3,
 193.4,
 193.5,
 193.6,
 193.7,
 193.8,
 193.9,
 194.0,
 194.1,
 194.2,
 194.3,
 194.4,
 194.5,
 194.6,
 194.7,
 194.8,
 194.9,
 195.0,
 195.1,
 195.2,
 195.3,
 195.4,
 195.5,
 195.6,
 195.7,
 195.8,
 195.9,
 196.0,
 196.1,
 196.2,
 196.3,
 196.4,
 196.5,
 196.6,
 196.7,
 196.8,
 196.9,
 197.0,
 197.1,
 197.2,
 197.3,
 197.4]

In [ ]:
# Assign each ask price to its corresponding price bin
# right=True means intervals are right-inclusive (opposite of bids)
# precision=10 ensures we don't lose decimal precision
asks_levels["bin"] = pd.cut(
    asks_levels['price'],
    bins=ask_levels_bounds,
    right=True,
    precision=10
)
asks_levels

,price,quantity,side,bin
0,187.25,74.361,ask,"(187.2, 187.3]"
1,187.26,3.222,ask,"(187.2, 187.3]"
2,187.27,17.383,ask,"(187.2, 187.3]"
3,187.28,229.780,ask,"(187.2, 187.3]"
4,187.29,299.143,ask,"(187.2, 187.3]"
...,...,...,...,...
995,197.20,33.770,ask,"(197.1, 197.2]"
996,197.21,0.620,ask,"(197.2, 197.3]"
997,197.22,1.638,ask,"(197.2, 197.3]"
998,197.23,8.418,ask,"(197.2, 197.3]"


In [ ]:
# Group the asks by their price bins and aggregate the data
# Similar to bids processing:
# - Sum quantities for each price level
# - Keep the 'side' information
# - Reset index for regular column access
asks_levels = asks_levels.groupby("bin").agg(
    quantity = ("quantity", "sum"),
    side = ("side", "first")
).reset_index()

# Extract the lower bound of each bin as the price level label
asks_levels['label'] = asks_levels['bin'].apply(lambda x: x.left)
asks_levels

C:\Users\Saksham Maggu\AppData\Local\Temp\ipykernel_2052\2917634651.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  asks_levels = asks_levels.groupby("bin").agg(


,bin,quantity,side,label
0,"(187.1, 187.2]",0.000,None,187.1
1,"(187.2, 187.3]",808.401,ask,187.2
2,"(187.3, 187.4]",3190.890,ask,187.3
3,"(187.4, 187.5]",3107.934,ask,187.4
4,"(187.5, 187.6]",2642.162,ask,187.5
...,...,...,...,...
98,"(196.9, 197.0]",2776.171,ask,196.9
99,"(197.0, 197.1]",84.650,ask,197.0
100,"(197.1, 197.2]",499.926,ask,197.1
101,"(197.2, 197.3]",21.189,ask,197.2


## Concatening both bid and ask in orderbook

In [ ]:
# Combine both bid and ask levels into a single orderbook DataFrame
# 1. Concatenate asks and bids
# 2. Remove any price levels with zero quantity
# 3. Sort by price (label) in descending order to show highest price first
orderbook = pd.concat([asks_levels, bids_levels])
orderbook = orderbook[orderbook['quantity'] > 0]
orderbook = orderbook.sort_values(by='label', ascending=False)
print(orderbook.to_string())

                bin   quantity side  label
101  (197.2, 197.3]     21.189  ask  197.2
100  (197.1, 197.2]    499.926  ask  197.1
99   (197.0, 197.1]     84.650  ask  197.0
98   (196.9, 197.0]   2776.171  ask  196.9
97   (196.8, 196.9]    110.337  ask  196.8
96   (196.7, 196.8]    175.472  ask  196.7
95   (196.6, 196.7]    503.400  ask  196.6
94   (196.5, 196.6]    278.435  ask  196.5
93   (196.4, 196.5]    523.609  ask  196.4
92   (196.3, 196.4]    335.383  ask  196.3
91   (196.2, 196.3]    524.681  ask  196.2
90   (196.1, 196.2]    463.340  ask  196.1
89   (196.0, 196.1]     82.231  ask  196.0
88   (195.9, 196.0]   1150.302  ask  195.9
87   (195.8, 195.9]    377.286  ask  195.8
86   (195.7, 195.8]     70.211  ask  195.7
85   (195.6, 195.7]    349.635  ask  195.6
84   (195.5, 195.6]    183.275  ask  195.5
83   (195.4, 195.5]    795.593  ask  195.4
82   (195.3, 195.4]     75.589  ask  195.3
81   (195.2, 195.3]    323.160  ask  195.2
80   (195.1, 195.2]    146.451  ask  195.1
79   (195.0